In [1]:
import os
import numpy as np
import tensorflow as tf
from keras.utils import to_categorical
from keras import models, layers
from keras.datasets import imdb
import tensorflow.keras.backend as K

2023-09-12 07:53:41.418273: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-12 07:53:41.455473: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-12 07:53:41.456089: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-12 07:53:42.076825: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
num_threads = 5
os.environ["OMP_NUM_THREADS"] = "5"
os.environ["TF_NUM_INTRAOP_THREADS"] = "5"
os.environ["TF_NUM_INTEROP_THREADS"] = "5"
tf.config.threading.set_inter_op_parallelism_threads(num_threads)
tf.config.threading.set_intra_op_parallelism_threads(num_threads)
tf.config.set_soft_device_placement(True)

In [3]:
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

In [4]:
print("The output categories are", np.unique(targets))
print("The number of unique words is", len(np.unique(np.hstack(data))))

The output categories are [0 1]
The number of unique words is 9998


In [5]:
length = [len(i) for i in data]
print("The Average Review length is", np.mean(length))
print("The Standard Deviation is", round(np.std(length)))

The Average Review length is 234.75892
The Standard Deviation is 173


In [6]:
# print("Label:", targets[0])
# print(data[0])
# print(len(data[0]), length[0])
# print(sum(data[0]))
# print(np.unique(data[0]))
# print(len(np.unique(data[0])))

In [11]:
index = imdb.get_word_index()
def vec2text(vec):
    reverse_index = dict([(value, key) for (key, value) in index.items()])
    decoded = " ".join( [reverse_index.get(i - 3, "#") for i in vec] )
    return decoded

def text2vector(text):
    v = []
    for t in text.split():
        n = index.get(t, None)
        if n is not None:
            v.append(n + 3)
        else:
            v.append(1)
    return v
    
# print(vec2text(data[0]))
# print(text2vector(vec2text(data[0])))

In [13]:
def vectorize(sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results

vdata = vectorize(data)
targets = np.array(targets).astype("float32")

In [14]:
test_x = vdata[:10000]
test_y = targets[:10000]
train_x = vdata[40000:]
train_y = targets[40000:]

In [15]:
model = models.Sequential()

2023-09-10 11:35:07.382759: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-10 11:35:07.383514: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [16]:
# Input - Layer

model.add(layers.Dense(50, activation = "relu", input_shape=(10000, )))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))

# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                500050    
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 505201 (1.93 MB)
Trainable params: 505201 (1.93 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [18]:
results = model.fit(train_x, train_y, epochs= 5, batch_size = 32, validation_data = (test_x, test_y))

2023-09-10 11:35:14.709373: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 400000000 exceeds 10% of free system memory.


Epoch 1/5
308/313 [============================>.] - ETA: 0s - loss: 0.3597 - accuracy: 0.8405

2023-09-10 11:35:18.290135: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 400000000 exceeds 10% of free system memory.


313/313 [==============================] - 4s 10ms/step - loss: 0.3592 - accuracy: 0.8406 - val_loss: 0.3184 - val_accuracy: 0.8657
Epoch 2/5
313/313 [==============================] - 2s 7ms/step - loss: 0.1391 - accuracy: 0.9472 - val_loss: 0.3791 - val_accuracy: 0.8609
Epoch 3/5
313/313 [==============================] - 2s 7ms/step - loss: 0.0616 - accuracy: 0.9782 - val_loss: 0.5450 - val_accuracy: 0.8449
Epoch 4/5
313/313 [==============================] - 2s 7ms/step - loss: 0.0280 - accuracy: 0.9905 - val_loss: 0.8221 - val_accuracy: 0.8375
Epoch 5/5
313/313 [==============================] - 2s 7ms/step - loss: 0.0284 - accuracy: 0.9900 - val_loss: 0.7493 - val_accuracy: 0.8469


In [19]:
scores = model.evaluate(test_x, test_y, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

2023-09-10 11:35:30.177269: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 400000000 exceeds 10% of free system memory.


Accuracy: 84.69%


In [20]:
tf.keras.saving.save_model(model, './data/keras-sentiment.mdl', overwrite=True, save_format=None)

INFO:tensorflow:Assets written to: ./data/keras-sentiment.mdl/assets


INFO:tensorflow:Assets written to: ./data/keras-sentiment.mdl/assets


In [21]:
my_model = tf.keras.saving.load_model('./data/keras-sentiment.mdl')
scores = my_model.evaluate(test_x, test_y, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

2023-09-10 11:35:33.273044: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 400000000 exceeds 10% of free system memory.


Accuracy: 84.69%


In [22]:
text = "# this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert # is an amazing actor and now the same being director # father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for # and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also # to the two little boy's that played the # of norman and paul they were just brilliant children are often left out of the # list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all"
sample_vector = vectorize([text2vector(text)])
prediction = my_model.predict(sample_vector)
print(prediction)

1/1 [==============================] - 0s 75ms/step
[[0.9998848]]


In [27]:
i = 100
print(targets[40000+i])
print(data[40000+i])
text = vec2text(data[40000+i])
print(text)

sample_vector = vectorize([text2vector(text)])
prediction = my_model.predict(sample_vector)
print(prediction)

0.0
[1, 167, 2979, 8513, 2733, 186, 8, 28, 2, 107, 275, 3167, 295, 159, 582, 56, 19, 14, 1403, 5675, 6, 212, 44, 9382, 5, 6, 2616, 44, 1937, 5, 2, 12, 1901, 11, 6, 1960, 22, 5988, 299, 6, 9257, 3071, 248, 1636, 2422, 37, 166, 369, 19, 2, 2, 1083, 5616, 2567, 5, 9, 515, 260, 3068, 19, 2567, 5, 41, 223, 64, 8, 1974, 2, 658, 2901, 2, 9, 2, 1003, 1586, 2733, 47, 6, 680, 570, 5, 380, 5704, 8, 27, 414, 63, 9, 1082, 821, 885, 1134, 43, 3437, 616, 88, 164, 674, 126, 186, 8, 79, 301, 2, 1523, 2567, 267, 2, 2, 11, 41, 2, 47, 6, 2, 1826, 21, 152, 2522, 17, 322, 5, 2, 214, 1571, 19, 6, 2, 2, 217, 240, 43, 50, 8, 30, 6, 2, 4, 20, 1027, 8, 1108, 32, 4, 2, 11, 6, 356, 420, 7, 2, 3219, 217, 11, 4, 2, 4, 248, 1636, 2, 4, 4192, 223, 132, 4, 780, 322, 5, 452, 37, 494, 53, 6, 3219, 359, 18, 668, 9528, 4254, 1540, 21, 164, 7407, 266, 46, 7, 134, 1008, 237, 2733, 64, 320, 2, 2, 4, 1341, 17, 6, 564, 29, 9, 1238, 879, 7, 2, 21, 9, 4170, 7, 6, 1288, 17, 73, 5, 4, 1185, 6519, 43, 214, 32, 2, 56, 300, 241, 39]
#

In [28]:
text = 'that was an awful movie i ever seen. worst than anything.' 
sample_vector = vectorize([text2vector(text)])
prediction = my_model.predict(sample_vector)
print(prediction)

1/1 [==============================] - 0s 16ms/step
[[0.00106448]]


In [29]:
text = 'never saw such an elegant plays in any other movies.' 
sample_vector = vectorize([text2vector(text)])
prediction = my_model.predict(sample_vector)
print(prediction)

1/1 [==============================] - 0s 19ms/step
[[0.9033039]]
